In [ ]:
M_sun = 1.9891*(10**33) #g
M_star = 1.2*M_sun
X = 0.70
Y = 0.27
Z = 1-X-Y

In [ ]:
#reading in the opacity table from OPAL
import pandas as pd
df = pd.read_excel("Book.xlsx", engine='openpyxl')

df.describe()

In [ ]:
import numpy as np

#numpy arrays are a bit easier to work with
df_np = df.to_numpy()
df_np.shape

In [ ]:
#adding labels for the default log(R) values, the 100 is to make the dimensions even but it should be ignored/not called
new_row = [100, -8.0, -7.5, -7.0, -6.5, -6.0, -5.5, -5.0, -4.5, -4.0, -3.5, -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0]
df_np = np.vstack([new_row, df_np])

In [ ]:
#converting from log(R) to log(rho) with our desired units (g cm−3)
log_R_array = np.array(new_row)
log_R_array = 10**(log_R_array-3)
log_rho_array = np.log10(log_R_array)
log_rho_array = log_rho_array[1:]
log_rho_array

In [ ]:
log_temp_array = df_np[1:,0]
log_temp_array

In [ ]:
#the initial interpolation range to fill in the missing, NaN values in the table.
interp_range = [-1.5,-1,-0.5,0,0.5,1,1.5,2,2.5,3]
df_np[1,1:]

In [ ]:
#the initial interpolation

test_table = []
for i in np.arange(1, 58):
    #test_table.append(df_np[i,1:20])
    PChip_test = scipy.interpolate.interp1d(log_rho_array,df_np[i,1:20], fill_value="extrapolate")
    test_table.append(np.concatenate((df_np[i,1:20],np.array(PChip_test(interp_range)))))

In [ ]:
#some checks to see if it looks fine
test_table = np.array(test_table)
test_table.shape

In [ ]:
#now have a table with regularly spaced intervals for the first 61 rows
np.savetxt('neat_opacity_table.txt',test_table)

In [ ]:
negative_check = []
for i in np.arange(61):
    negative_check.append(test_table[i,28]-test_table[i,15])

In [ ]:
#this is to check that the interpolation works correctly, since opacity values should only increasing density
#and most easily accessible python interpolation schemes require monotonically increasing functions
negative_check[:]

In [ ]:
np.where(np.array(negative_check) < 0)

In [ ]:
#some failed attempts with different interpolation schemes before ultimately arriving at interp1d from scipy

#PChip_test = scipy.interpolate.PchipInterpolator(log_rho_array,df_np[10,1:])
#PChip_test = scipy.interpolate.CubicSpline(log_rho_array,df_np[10,1:])
#PChip_test = scipy.interpolate.Akima1DInterpolator(log_rho_array,df_np[10,1:])
PChip_test = scipy.interpolate.interp1d(log_rho_array,df_np[10,1:], fill_value="extrapolate")
PChip_test(interp_range)

In [ ]:
#now manually doing the last few rows with NAN values for higher densities. These are manually done because 
#I found it much faster than trying to automate for inconsistent dimensions

#this corresponds to log(T) = 7.2
interp_range = [-2,-1.5,-1,-0.5,0,0.5,1,1.5,2,2.5,3]
PChip_test_manual_1 = scipy.interpolate.interp1d(log_rho_array[:-1],df_np[58,1:-1], fill_value="extrapolate")
test_table = np.vstack((test_table,(np.concatenate((df_np[58,1:-1],np.array(PChip_test_manual_1(interp_range)))))))

In [ ]:
test_table.shape

In [ ]:
#corresponds to log(T) = 7.3
interp_range = [-2.5,-2,-1.5,-1,-0.5,0,0.5,1,1.5,2,2.5,3]
PChip_test_manual_2 = scipy.interpolate.interp1d(log_rho_array[:-2],df_np[59,1:-2], fill_value="extrapolate")
test_table = np.vstack((test_table,(np.concatenate((df_np[59,1:-2],np.array(PChip_test_manual_2(interp_range)))))))

In [ ]:
#corresponds to log(T) = 7.4
interp_range = [-2.5,-2,-1.5,-1,-0.5,0,0.5,1,1.5,2,2.5,3]
PChip_test_manual_3 = scipy.interpolate.interp1d(log_rho_array[:-2],df_np[60,1:-2], fill_value="extrapolate")
test_table = np.vstack((test_table,(np.concatenate((df_np[60,1:-2],np.array(PChip_test_manual_3(interp_range)))))))

In [ ]:
#corresponds to log(T) = 7.5
interp_range = [-3, -2.5,-2,-1.5,-1,-0.5,0,0.5,1,1.5,2,2.5,3]
PChip_test_manual_4 = scipy.interpolate.interp1d(log_rho_array[:-3],df_np[61,1:-3], fill_value="extrapolate")
test_table = np.vstack((test_table,(np.concatenate((df_np[61,1:-3],np.array(PChip_test_manual_4(interp_range)))))))

In [ ]:
interp_range_log_rho_array = [-1.5,-1,-0.5,0,0.5,1,1.5,2,2.5,3]
interp_range_log_rho_array = np.array(interp_range_log_rho_array)
log_rho_array = np.concatenate((log_rho_array,interp_range_log_rho_array))

In [ ]:
log_temp_array = log_temp_array[0:61]
log_temp_array.shape

In [ ]:
#we now have updated arrays for the two axes for density and temperature, in addition to interpolated values
#for our required range
combined_log_temp_rho_array = []
test_grid = scipy.interpolate.RegularGridInterpolator((log_temp_array,log_rho_array),test_table)

In [ ]:
def kappa(T,rho):
    #T needs to be in K and rho needs to be in g*cm^-3
    #works in the required interval of [3.75<log10(T/K)<7.5] and [−9<log10(ρ/g*cm^−3)<3]
    logT = np.log10(T)
    logrho = np.log10(rho)
    return test_grid(logT,logrho)

## Everything above this cell is from interpolating the opacity tables

In [ ]:
#cell of constants

#stefan_boltzmann constant in cgs units
sb_constant = 5.670374419*10**-5
c = 2.998*10**10  #cm/s
a = 4*sb_constant/c
G = 6.67430*10**-8

In [ ]:
#function of pp-chains energy generation in units of erg/g/s
#inputs have unit requirements. rho must be in g/cm**3, T must be in K
def epsilon_pp(rho,T,X):
    return (2.4*10**6)*(X**2)*rho*((T/10**6)**(-2/3))*np.exp(-33.8/(T/10**6)**(1/3))

In [ ]:
def epsilon_CNO(rho,T,X,Y):
    Z = 1-X-Y
    X_CNO = Z/2
    return (8.7*10**27)*X*X_CNO*rho*((T/10**6)**(-2/3))*np.exp(-152/(T/10**6)**(1/3))

In [ ]:
def rho(P,T,X):
    #stefan_boltzmann constant in cgs units
    sb_constant = 5.670374419*10**-5
    c = 2.998*10**10  #cm/s
    a = 4*sb_constant/c
    
    P_rad = (1/3)*a*T**4
    P_gas = P-P_rad
    #if something weird happens while converging
    if P_gas <= 0 or T<=0:
        P_gas = P
        print('eh youre getting some negative values for P_gas or T')
    
    mu = 4/(3+5*X)
    m_H = 1.6738*10**-24  #in grams
    k_B = 1.380649*10**-16  #in cgs units
    
    return (P_gas*mu*m_H)/(k_B*T)

In [ ]:
def dMdr(r,M):
    a = r*2
    rho = M/(r+a)**2
    return 4*np.pi*rho*r**2


def rho_sample_1(r,M):
    G = 6.67430*10**-8
    return M/(r)**2

#dMdr(r,rho_sample_1(r,M)) where M=1.2 and we wanna leave r as r

In [ ]:
#remember, load is in the form [L,P,R,T]
def dLdM(M,L):
    X = 0.7
    P = current_parameters[1]
    T = current_parameters[3]
    return (epsilon_pp(rho(P,T,X),T,X) + epsilon_CNO(rho(P,T,X),T,X,Y))

In [ ]:
def dPdM(M,P):
    R = current_parameters[2]
    return (-G*M)/(4*np.pi*R**4)

In [ ]:
def dRdM(M,R):
    X = 0.7
    P = current_parameters[1]
    T = current_parameters[3]
    rh = rho(P,T,X)
    return 1/(4*np.pi*rh*R**2)

In [ ]:
#dlnPdlnT(P,P_previous,T,T_previous)
def dTdM(M,T):
    R = current_parameters[2]
    P = current_parameters[1]
    T = current_parameters[3]
    
    P_prev = previous_parameters[1]
    T_prev = previous_parameters[3]
    
    temp_grad = dlnPdlnT(P,P_prev,T,T_prev)
    
    return -G*M*temp_grad*T*(2/5)/(4*np.pi*P*R**4)

In [ ]:
#will use this for the first inwards shell
def dTdM_s(M,T):
    P = load2[1]
    R = load2[2]
    return -G*M*T*(2/5)/(4*np.pi*P*R**4)

In [ ]:
def dTdM_core_rad(M,T):
    P = previous_parameters_out[1]
    R = previous_parameters_out[2]
    return -G*M*T*(adiabatic_grad())/(4*np.pi*P*R**4)

In [ ]:
def derivs(M,L,P,R,T):
    derivs_array[0] = dLdM(M,L)
    derivs_array[1] = dPdM(M,P)
    derivs_array[2] = dRdM(M,R)
    derivs_array[3] = dTdM(M,T)
    return derivs_array

In [ ]:
yet_another_test = []
for i in [1,2,3,4]:
    derivs()
    inwards_sol = scipy.integrate.solve_ivp(derivs[i],[1,M_sun],load1)
    yet_another_test.append(inwards_sol)

In [ ]:
inwards_sol_L = scipy.integrate.solve_ivp(dLdM_s,(M_star,M_sun),[load2[0]], t_eval = [M_sun])
inwards_sol_P = scipy.integrate.solve_ivp(dPdM_s,(M_star,M_sun),[load2[1]], t_eval = [M_sun])
inwards_sol_R = scipy.integrate.solve_ivp(dRdM_s,(M_star,M_sun),[load2[2]], t_eval = [M_sun])
#inwards_sol_T = scipy.integrate.solve_ivp(dTdM_s,(M_star,M_sun),[load2[3]], t_eval = [M_sun])

In [ ]:
outwards_sol_L = scipy.integrate.solve_ivp(dLdM_c,(1,M_sun),[load1[0]], t_eval = [M_sun])
outwards_sol_P = scipy.integrate.solve_ivp(dPdM_c,(1,M_sun),[load1[1]], t_eval = [M_sun])
outwards_sol_R = scipy.integrate.solve_ivp(dRdM_c,(1,M_sun),[load1[2]], t_eval = [M_sun])
#outwards_sol_T = scipy.integrate.solve_ivp(dTdM_c,(1,M_sun),[load1[3]], t_eval = [M_sun])

In [ ]:
inwards_parameter_update = [inwards_sol_L.y, inwards_sol_P.y, inwards_sol_R.y]
inwards_parameter_update = np.array(inwards_parameter_update)

outwards_parameter_update = [outwards_sol_L.y, outwards_sol_P.y, outwards_sol_R.y]
outwards_parameter_update = np.array(outwards_parameter_update)

difference_array = inwards_parameter_update - outwards_parameter_update
difference_array

In [ ]:
inwards_parameter_update

In [ ]:
outwards_parameter_update.shape

In [ ]:
#make a print statement for each iteration so that I can see the time it takes per iteration

#figuring out step size for initial value adjustment. Want to set it to go both negative and positive
#if load1[0]-prev_load1[0]

if (difference_array[0] - prev_difference_array[0]) > 0:
    x == -1
if (difference_array[0] - prev_difference_array[0]) <= 0:
    x == x
load1[0] = prev_load1[0] + x*prev_load1[0]*0.01

iteration_count.append(iteration_count + 1)

#my condition for stopping? 
if difference_array[0] < 1000 or difference_array[1] < 1000 or difference_array[2] < 1000:
    break
    
    
#load1[0] = load1[0] + (0.01*load1[0])

#load1[0] = outwards_parameter_update[0]
#load2[0] =

In [ ]:
iteration_count = 0
for i in np.linspace(1,100,100):
    prev_load1 = load1
    prev_load2 = load2
    prev_difference_array = difference_array
    
    inwards_sol_L = scipy.integrate.solve_ivp(dLdM_s,(M_star,M_sun),[load2[0]], t_eval = [M_sun])
    inwards_sol_P = scipy.integrate.solve_ivp(dPdM_s,(M_star,M_sun),[load2[1]], t_eval = [M_sun])
    inwards_sol_R = scipy.integrate.solve_ivp(dRdM_s,(M_star,M_sun),[load2[2]], t_eval = [M_sun])

    outwards_sol_L = scipy.integrate.solve_ivp(dLdM_c,(1,M_sun),[load1[0]], t_eval = [M_sun])
    outwards_sol_P = scipy.integrate.solve_ivp(dPdM_c,(1,M_sun),[load1[1]], t_eval = [M_sun])
    outwards_sol_R = scipy.integrate.solve_ivp(dRdM_c,(1,M_sun),[load1[2]], t_eval = [M_sun])

    #these are the updated results for L(x_f), P(x_f), and R(x_f) integrating from the surface inwards
    inwards_parameter_update = [inwards_sol_L.y, inwards_sol_P.y, inwards_sol_R.y]
    inwards_parameter_update = np.array(inwards_parameter_update)

    #these are the updated results for L(x_f), P(x_f), and R(x_f) integrating from the core outwards
    outwards_parameter_update = [outwards_sol_L.y, outwards_sol_P.y, outwards_sol_R.y]
    outwards_parameter_update = np.array(outwards_parameter_update)

    #how close to converging are my integrations from both ends?
    difference_array = abs(inwards_parameter_update - outwards_parameter_update)

    
    #moving in the incorrect direction, let's change the sign
    if (difference_array[1] - prev_difference_array[1]) > 0:
        x = -1
    if (difference_array[1] - prev_difference_array[1]) <= 0:
        x = x
    #new guess for P_c
    load1[1] = prev_load1[1] + x*prev_load1[1]*0.001
    
    
    #moving in the incorrect direction, let's change the sign
    if (difference_array[0] - prev_difference_array[0]) > 0:
        y = -1
    if (difference_array[0] - prev_difference_array[0]) <= 0:
        y = y
    #new guess for L_s
    load2[0] = prev_load2[0] + y*prev_load2[0]*0.001
    
    
    #moving in the incorrect direction, let's change the sign
    if (difference_array[2] - prev_difference_array[2]) > 0:
        z = -1
    if (difference_array[2] - prev_difference_array[2]) <= 0:
        z = z
    #new guess for R_s
    load2[2] = prev_load2[2] + z*prev_load2[2]*0.001
    
    iteration_count = i
    print(iteration_count)
    
#iteration_count.append(iteration_count + 1)

#my condition for stopping? 
    #if difference_array[0] < 10 or difference_array[1] < 10 or difference_array[2] < 10:
        #break
   

In [ ]:
def adiabatic_grad():
    #stefan_boltzmann constant in cgs units
    sb_constant = 5.670374419*10**-5
    c = 2.998*10**10  #cm/s
    a = 4*sb_constant/c
    
    M = M_prev
    L = previous_parameters_out[0]
    T = previous_parameters_out[3]
    P = previous_parameters_out[1]
    rh = rho(P,T,0.7)
    k = kappa(T,rh)
    
    return (3/(16*np.pi*a*c))*((P*k*L)/(G*M*T**4))

In [ ]:
#load1 corresponds to core initial values in the form [L,P,r,T]
load1 = [0.0001,1.897e17,0.0001,16.67e6]
#load2 is the same but for surface initial values with L in erg/s
load2 = [8.4156*10**33,0.0001,8.627*10**10,0.0001]
sample_test_2_solutions = scipy.integrate.solve_ivp(dLdM,[0.1,1.2],load_test_1, t_eval = [0.1,0.5,1.2])

In [ ]:
load_test = np.split(load1,[0])
load_test[1]

In [ ]:
dLdM(load1[1],load1[3],0.7)

In [ ]:
sample_test_2_solutions

In [ ]:
surface_bc_T_P = []
def surface_boundry_conditions_temp_pressure(M,L,R):
    P = (2*G*M)/(3*R**2)
    T = (L/(4*np.pi*sb_constant*R**2))**(0.25)
    return np.array([P,T])

#the idea is that you use L and R initial guesses from load2 as arguments, and you update load2 values for P and T

In [ ]:
#honestly, I think my initial luminosity guess was wrong so updating it here to stellar lum as a place to start
load2[0] = 2e35

surf_bc = surface_boundry_conditions_temp_pressure(M_star,load2[0],load2[2])
#update surface Pressure
load2[1] = surf_bc[0]
#update surface Temp
load2[3] = surf_bc[1]

#remember, load is in the form [L,P,R,T]

In [ ]:
def dlnPdlnT(P,P_previous,T,T_previous):
    if P_previous < 0:
        print('P_prev below zero')
    if P < 0:
        print('P below zero')
    if T_previous < 0:
        print('T_prev below zero')
    if T < 0:
        print('T below zero')
        
    if (T_previous-T) == 0:
        print('T_previous-T is zero')
        
    if (P_previous+P) < 0:
        print('P_previous+P below zero')
    
    return ((P_previous-P)/(T_previous-T))*((T_previous+T)/(P_previous+P))

In [ ]:
#og
#dlnPdlnT(P,P_previous,T,T_previous)
def dTdM(M,T):
    R = current_parameters[2]
    P = current_parameters[1]
    T = current_parameters[3]
    
    P_prev = previous_parameters[1]
    T_prev = previous_parameters[3]
    
    temp_grad = dlnPdlnT(P,P_prev,T,T_prev)
    
    return -G*M*temp_grad*T*(2/5)/(4*np.pi*P*R**4)

In [ ]:
load2
previous_parameters = load2
previous_parameters

In [ ]:
#let me set up a few rounds of integration but for all 4 functions, since we need the parameter values for inputs
dM = -M_star / 100
total_steps = -M_star/dM

previous_parameters = load2
#just for the first inward shell, set current = previous
current_parameters = previous_parameters
M_prev = M_star

first_shell_in_L = scipy.integrate.solve_ivp(dLdM,(M_prev,M_prev + dM),[previous_parameters[0]], t_eval = [M_prev + dM])
first_shell_in_P = scipy.integrate.solve_ivp(dPdM,(M_prev,M_prev + dM),[previous_parameters[1]], t_eval = [M_prev + dM])
first_shell_in_R = scipy.integrate.solve_ivp(dRdM_s,(M_prev,M_prev + dM),[previous_parameters[2]], t_eval = [M_prev + dM])
first_shell_in_T = scipy.integrate.solve_ivp(dTdM_s,(M_prev,M_prev + dM),[previous_parameters[3]], t_eval = [M_prev + dM])

current_parameters = [first_shell_in_L.y, first_shell_in_P.y, first_shell_in_R.y, first_shell_in_T.y]
current_parameters = np.array(current_parameters)

M_prev = M_prev + dM

parameters_table_1 = []
print(first_shell_in_L.t, first_shell_in_P.t, first_shell_in_R.t, first_shell_in_T.t)
print(current_parameters[3]-previous_parameters[3])

In [ ]:
checking_masses_array = []
for i in np.linspace(1,100,100):
    
    inwards_sol_L_test1 = scipy.integrate.solve_ivp(dLdM,(M_prev,M_prev + dM),current_parameters[0,0], t_eval = [M_prev + dM])
    inwards_sol_P_test1 = scipy.integrate.solve_ivp(dPdM,(M_prev,M_prev + dM),current_parameters[1,0], t_eval = [M_prev + dM])
    inwards_sol_R_test1 = scipy.integrate.solve_ivp(dRdM_s,(M_prev,M_prev + dM),current_parameters[2,0], t_eval = [M_prev + dM])
    inwards_sol_T_test1 = scipy.integrate.solve_ivp(dTdM,(M_prev,M_prev + dM),current_parameters[3,0], t_eval = [M_prev + dM])
    
    previous_parameters = current_parameters.copy()
    
    L_est = previous_parameters[0] + dLdM(M_prev,previous_parameters[0])*dM
    P_est = previous_parameters[1] + dPdM(M_prev,previous_parameters[1])*dM
    R_est = previous_parameters[2] + dRdM(M_prev,previous_parameters[2])*dM
    T_est = previous_parameters[3] + dTdM(M_prev,previous_parameters[3])*dM
    
    
    #if abs((L_est-inwards_sol_L_test1.y)/L_est) > 0.001:
    if i>4:
        current_parameters[0] = L_est
    else:
        current_parameters[0] = inwards_sol_L_test1.y
        
        
    #if abs((P_est-inwards_sol_P_test1.y)/P_est) > 0.001:
    if i>4:
        current_parameters[1] = P_est
    else:
        current_parameters[1] = inwards_sol_P_test1.y
        
        
    #if abs((R_est-inwards_sol_R_test1.y)/R_est) > 0.001:
    if i>4:
        current_parameters[2] = R_est
    else:
        current_parameters[2] = inwards_sol_R_test1.y
        
        
    #if ((T_est-inwards_sol_T_test1.y)/T_est) > 0.01:
    if i>4:
        current_parameters[3] = T_est
    else:
        current_parameters[3] = inwards_sol_T_test1.y
        
    #current_parameters = [inwards_sol_L_test1.y, inwards_sol_P_test1.y, inwards_sol_R_test1.y, inwards_sol_T_test1.y]
    #just trying shit at this point
    current_parameters = [L_est,P_est,R_est,T_est]
    current_parameters = np.array(current_parameters)
    M_prev = M_prev + dM
    
    
    
    parameters_table_1.append(current_parameters)
    checking_masses_array.append(inwards_sol_L_test1.t)
    
    print(i)

In [ ]:
T_est = previous_parameters[3] + dTdM(M_prev,previous_parameters[3])*dM
print(dTdM(M_prev,previous_parameters[3])*dM)

In [ ]:
#let me set up a few rounds of integration but for all 4 functions, since we need the parameter values for inputs
dM_out = (0.8*M_sun)/ 100
#total_steps = M_star/dM

previous_parameters_out = load1
#just for the first inward shell, set current = previous
current_parameters_out = previous_parameters_out
M_prev = 1

#first_shell_out_L = scipy.integrate.solve_ivp(dLdM,(M_prev,M_prev + dM),[previous_parameters_out[0]], t_eval = [M_prev + dM])
#first_shell_out_P = scipy.integrate.solve_ivp(dPdM,(M_prev,M_prev + dM),[previous_parameters_out[1]], t_eval = [M_prev + dM])
#first_shell_out_R = scipy.integrate.solve_ivp(dRdM_s,(M_prev,M_prev + dM),[previous_parameters_out[2]], t_eval = [M_prev + dM])
first_shell_out_T = scipy.integrate.solve_ivp(dTdM_core_rad,(M_prev,M_prev + dM),[previous_parameters_out[3]], t_eval = [M_prev + dM])

current_parameters_out = [first_shell_out_L.y, first_shell_out_P.y, first_shell_out_R.y, first_shell_out_T.y]
current_parameters_out = np.array(current_parameters_out)

M_prev = M_prev + dM_out

parameters_table_1 = []
print(first_shell_out_L.t, first_shell_out_P.t, first_shell_out_R.t, first_shell_out_T.t)

In [ ]:
#in the form [L,P,R,T]
load1

In [ ]:
checking_masses_array = []
for i in np.linspace(1,100,100):
    
    inwards_sol_L_test1 = scipy.integrate.solve_ivp(dLdM,(M_prev,M_prev + dM),current_parameters[0,0], t_eval = [M_prev + dM])
    inwards_sol_P_test1 = scipy.integrate.solve_ivp(dPdM,(M_prev,M_prev + dM),current_parameters[1,0], t_eval = [M_prev + dM])
    inwards_sol_R_test1 = scipy.integrate.solve_ivp(dRdM_s,(M_prev,M_prev + dM),current_parameters[2,0], t_eval = [M_prev + dM])
    inwards_sol_T_test1 = scipy.integrate.solve_ivp(dTdM,(M_prev,M_prev + dM),current_parameters[3,0], t_eval = [M_prev + dM])
    
    previous_parameters = current_parameters.copy()
    
    L_est = previous_parameters[0] + dLdM(M_prev,previous_parameters[0])*dM
    P_est = previous_parameters[1] + dPdM(M_prev,previous_parameters[1])*dM
    R_est = previous_parameters[2] + dRdM(M_prev,previous_parameters[2])*dM
    T_est = previous_parameters[3] + dTdM(M_prev,previous_parameters[3])*dM
    
    
    #if abs((L_est-inwards_sol_L_test1.y)/L_est) > 0.001:
    if i>4:
        current_parameters[0] = L_est
    else:
        current_parameters[0] = inwards_sol_L_test1.y
        
        
    #if abs((P_est-inwards_sol_P_test1.y)/P_est) > 0.001:
    if i>4:
        current_parameters[1] = P_est
    else:
        current_parameters[1] = inwards_sol_P_test1.y
        
        
    #if abs((R_est-inwards_sol_R_test1.y)/R_est) > 0.001:
    if i>4:
        current_parameters[2] = R_est
    else:
        current_parameters[2] = inwards_sol_R_test1.y
        
        
    #if ((T_est-inwards_sol_T_test1.y)/T_est) > 0.01:
    if i>4:
        current_parameters[3] = T_est
    else:
        current_parameters[3] = inwards_sol_T_test1.y
        
    #current_parameters = [inwards_sol_L_test1.y, inwards_sol_P_test1.y, inwards_sol_R_test1.y, inwards_sol_T_test1.y]
    #just trying shit at this point
    current_parameters = [L_est,P_est,R_est,T_est]
    current_parameters = np.array(current_parameters)
    M_prev = M_prev + dM
    
    
    
    parameters_table_1.append(current_parameters)
    checking_masses_array.append(inwards_sol_L_test1.t)
    
    print(i)

## Everything above this cell are attempts at using scipy. Below, I'm trying to do it myself

In [ ]:
#TABLE # 74     $G&N'93 Solar$     X=0.7000 Y=0.2700 Z=0.0300 dXc=0.0000 dXo=0.0000
#cell of constants

#stefan_boltzmann constant in cgs units
sb_constant = 5.670374419e-5
c = 2.998e10  #cm/s
a = 4*sb_constant/c
G = 6.67430e-8


M_sun = 1.9891e33 #g
M_star = 1.2*M_sun
X = 0.70
Y = 0.27
Z = 1-X-Y

In [ ]:
def kappa(T,rho):
    #T needs to be in K and rho needs to be in g*cm^-3
    #works in the required interval of [3.75<log10(T/K)<7.5] and [−9<log10(ρ/g*cm^−3)<3]
    logT = np.log10(T)
    logrho = np.log10(rho)
    return test_grid(logT,logrho)

In [ ]:
#function of pp-chains energy generation in units of erg/g/s
#inputs have unit requirements. rho must be in g/cm**3, T must be in K
#6.76 approximation 
def epsilon_pp(rho,T,X):
    return (2.4e4)*(X**2)*rho*((T/10**9)**(-2/3))*np.exp(-3.38/(T/10**9)**(1/3))

#6.77, where CNO mass fraction is estimated to be Z/2
def epsilon_CNO(rho,T,X,Y):
    Z = 1-X-Y
    return (4.4e25)*X*Z*rho*((T/10**9)**(-2/3))*np.exp(-15.228/(T/10**9)**(1/3))


In [ ]:
def rho(P,T,X):
    P_rad = (1/3)*a*T**4
    P_gas = P-P_rad
    #if something weird happens while converging
    if P_gas <= 0 or T<=0:
        P_gas = P
        print('eh youre getting some negative values for P_gas or T')
    
    mu = 4/(3+5*X)
    m_H = 1.6738e-24  #in grams
    k_B = 1.380649e-16  #in cgs units
    
    return (P_gas*mu*m_H)/(k_B*T)

In [ ]:
def dLdM(P,T):
    X = 0.7
    dens = rho(P,T,X)
    return (epsilon_pp(dens,T,X) + epsilon_CNO(dens,T,X,Y))



def dPdM(M,R):
    return -(G*M)/(4*np.pi*R**4)



def dRdM(P,T,R):
    X = 0.7
    dens = rho(P,T,X)
    return 1/(4*np.pi*dens*R**2)



def dlnPdlnT(P,P_previous,T,T_previous):
    if P_previous < 0:
        print('P_prev below zero')
    if P < 0:
        print('P below zero')
    if T_previous < 0:
        print('T_prev below zero')
    if T < 0:
        print('T below zero')
        
    if (T_previous-T) == 0:
        print('T_previous-T is zero')
        
    if (P_previous+P) < 0:
        print('P_previous+P below zero')
    
    return ((P_previous-P)/(T_previous-T))*((T_previous+T)/(P_previous+P))



def dTdM(P,T,R,M,temp_grad):
    return -G*M*temp_grad*T/(4*np.pi*P*R**4)


def derivs(P,T,R,M,L,temp_grad):
    dldm = dLdM(P,T)
    dpdm = dPdM(M,R)
    drdm = dRdM(P,T,R)
    dtdm = dTdM(P,T,R,M,temp_grad)
    interm = [dldm, dpdm, drdm, dtdm]
    return np.array(interm)

In [ ]:
#load1 corresponds to core initial values in the form [L,P,r,T]
load1 = [0.0001,2.68e8,0.0001,16.67e6]
#load2 is the same but for surface initial values with L in erg/s
load2 = [8.4156e33,0.0001,8.627e10,0.0001]


surface_bc_T_P = []
def surface_boundry_conditions_temp_pressure(M,L,R):
    P = (2*G*M)/(3*R**2)
    T = (L/(4*np.pi*sb_constant*R**2))**(0.25)
    return np.array([P,T])

#the idea is that you use L and R initial guesses from load2 as arguments, and you update load2 values for P and T

In [ ]:
surf_bc = surface_boundry_conditions_temp_pressure(M_star,load2[0],load2[2])
#update surface Pressure
load2[1] = surf_bc[0]
#update surface Temp
load2[3] = surf_bc[1]

#remember, load is in the form [L,P,R,T]
print(load2)
stipulation = np.array(load2) - np.array(load1)
print(stipulation)

In [ ]:
table_parameters = []
dM_in = -(M_star - 0.8*M_sun)/1000
t_grad_ini = 2/5

previous_parameters = load2
ddm = derivs(previous_parameters[1], previous_parameters[3], previous_parameters[2], M_star, previous_parameters[0], t_grad_ini)

current_parameters = previous_parameters + ddm*dM_in

#if (current_parameters[0]-previous_parameters[0])/previous_parameters[0] > 0.01:
#    current_parameters[0] = 0.01


In [ ]:
table_parameters = []
dM_out = (0.8*M_sun-1)/1000
t_grad_ini = 2/5

previous_parameters = load1
ddm = derivs(previous_parameters[1], previous_parameters[3], previous_parameters[2], M_star, previous_parameters[0], t_grad_ini)

current_parameters = previous_parameters + ddm*dM_in

M_current = M_star + dM_out


table_parameters.append(current_parameters)
#if (current_parameters[0]-previous_parameters[0])/previous_parameters[0] > 0.01:
#    current_parameters[0] = 0.01

for i in np.linspace(1,1000,1000):
    previous_parameters = current_parameters
    ddm = derivs(previous_parameters[1], previous_parameters[3], previous_parameters[2], M_current, previous_parameters[0], t_grad_ini)
    current_parameters = previous_parameters + ddm*dM_out
    table_parameters.append(current_parameters)
    M_current = M_current + dM_out
    print(i)

In [ ]:
table_parameters